In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sc
#from scipy import io
#from collections import Counter
from matplotlib import pyplot as plt
from signet.cluster import Cluster
from signet.block_models import SSBM
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

In [16]:
df1 = pd.read_csv("soc-sign-epinions.csv")
df1 = df1.drop_duplicates(inplace = False)
df1

,source,target,sign
0,1,2,-1
1,2,128553,-1
2,3,4,1
3,5,6,-1
4,5,156,-1
...,...,...,...
841367,131822,367,1
841368,131823,112102,1
841369,131824,131825,1
841370,131826,131827,1


In [17]:
train, test, train_label, test_label = train_test_split(df1, df1['source'], test_size = 0.7)

In [141]:
#train1, test1, train_label1, test_label1 = train_test_split(train, train['source'], test_size = 0.3)

In [18]:
G = nx.DiGraph()
G = nx.from_pandas_edgelist(df=train,source='source',target='target',edge_attr='sign',create_using=nx.DiGraph(),edge_key='sign')

In [4]:
#nx.draw_networkx(G, with_labels=True,node_color="red",edge_color='blue',font_size=10,node_size=280)

In [19]:
print('number of nodes G =',nx.number_of_nodes(G) )
print('number of edges G =',nx.number_of_edges(G) )
print('Density of G:', nx.density(G))


number of nodes G = 71880
number of edges G = 252411
Density of G: 4.885378114458306e-05


In [49]:
G.nodes()

NodeView((112893, 29267, 3364, 905, 75856, 55386, 37400, 58, 2764, 3211, 80, 57057, 12137, 16508, 11673, 11874, 5455, 77066, 34110, 12148, 30900, 56852, 20273, 105674, 93303, 550, 12986, 1402, 66502, 4949, 3245, 3415, 111768, 3751, 24568, 13096, 5759, 879, 3170, 5106, 2192, 11773, 72165, 72272, 3992, 22180, 51313, 59979, 91588, 91589, 9054, 43107, 6837, 42517, 9220, 4989, 5699, 26, 12169, 71950, 37985, 6307, 17750, 97890, 23033, 667, 125010, 1686, 54434, 57152, 16318, 4901, 1076, 17043, 669, 57166, 56004, 66664, 33810, 9020, 17067, 33934, 7339, 60147, 44873, 11542, 16106, 15463, 20896, 4690, 8931, 1512, 18840, 24184, 122172, 1442, 4666, 11852, 60145, 1683, 111393, 50501, 70326, 945, 13864, 12966, 637, 121277, 92838, 1439, 6301, 5983, 314, 10429, 125941, 16565, 58229, 21643, 57077, 47208, 3884, 7645, 96565, 36700, 55381, 8969, 24102, 103655, 1334, 5567, 21278, 5948, 23562, 46073, 6365, 26987, 14654, 32129, 34580, 107662, 1346, 11729, 47946, 17690, 12562, 1031, 3250, 63711, 23533, 109801

In [20]:
AA1=nx.get_edge_attributes(G,'sign')  # مربوط به گراف دوم است attribute
AA1[(1,2)]

In [21]:
rows = train.loc[:, "source"].to_numpy()
cols = train.loc[:, "target"].to_numpy()
sign = train.loc[:, "sign"].to_numpy()

# این قسمت تست اون مثال کوچک استاد است با استفاده از ایجاد ماتریس شباهت

In [10]:

#p_rows = np.array([1,1,2,2,3,3,4,4,5,5,6,6])
#p_cols = np.array([2,3,1,3,1,2,5,6,4,6,4,5])
#p_data = np.ones(len(p_rows))

#n_rows = np.array([1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6])
#n_cols = np.array([4,5,6,4,5,6,4,5,6,1,2,3,1,2,3,1,2,3])
#n_data = np.ones(len(n_rows))

In [22]:
#A_p = sc.sparse.csc_matrix((p_data, (p_rows, p_cols)), shape = (7,7))
#A_n = sc.sparse.csc_matrix((n_data, (n_rows, n_cols)), shape = (7,7))
#print("A_p",A_p)
#print("A_n",A_n)

In [23]:
#print(p_data.shape)
#print(n_data.shape)

In [24]:
p_rows = [rows[i] for i in range(len(rows)) if sign[i] == 1]
p_cols = [cols[i] for i in range(len(cols)) if sign[i] == 1]
p_data = np.ones(len(p_rows))

n_rows = [rows[i] for i in range(len(rows)) if sign[i] == -1]
n_cols = [cols[i] for i in range(len(cols)) if sign[i] == -1]
n_data = np.ones(len(n_rows))

In [25]:
A_p = sc.sparse.csc_matrix((p_data, (p_rows, p_cols)), shape = (131829,131829))
A_n = sc.sparse.csc_matrix((n_data, (n_rows, n_cols)), shape = (131829,131829))
A_p = A_p+A_p.T
#A_n = A_n+A_n.T
#print("A_p",A_p)
#print("A_n",A_n)

In [26]:
print(p_data.shape)
print(n_data.shape)

(215386,)
(37025,)


In [27]:
c = Cluster((A_p, A_n))

In [29]:
    AAA=c.spectral_cluster_bnc(k=5, normalisation='sym', eigens=None, mi=None)
    BBB=c.spectral_cluster_laplacian(k=5, normalisation='sym_sep', eigens=None, mi=None)
print(AAA)

[0 2 1 ... 2 2 4]


In [15]:
AAA = np.zeros((20, 7))
#BBB = np.zeros((20, 6))

for i in range(20):
    AAA[i]=c.spectral_cluster_bnc(k=2, normalisation='sym', eigens=None, mi=None)
    #BBB[i]=c.spectral_cluster_laplacian(k=5, normalisation='sym_sep', eigens=None, mi=None)
print(AAA)

[[1. 0. 0. 0. 1. 1. 1.]
 [1. 0. 0. 0. 1. 1. 1.]
 [1. 0. 0. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 1. 1.]
 [1. 0. 0. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 0. 0.]]


In [30]:
cluster1=[i for i in G.nodes() if AAA[i] == 0]
cluster2=[i for i in G.nodes() if AAA[i] == 1]
cluster3=[i for i in G.nodes() if AAA[i] == 2]
cluster4=[i for i in G.nodes() if AAA[i] == 3]
cluster5=[i for i in G.nodes() if AAA[i] == 4]
#main_clusters = np.array([cluster1, cluster2], dtype = np.ndarray)
main_clusters = np.array([cluster1, cluster2, cluster3, cluster4, cluster5], dtype = np.ndarray)

In [168]:
print("cluster1",cluster1)
print("cluster2",cluster4)


cluster1 [55915, 20825, 124923, 13733, 8862, 38635, 54597, 688, 168, 3168, 4170, 108343, 108344, 60013, 55836, 29907, 4575, 322, 11680, 2374, 931, 33965, 27130, 46721, 1096, 24420, 26573, 17397, 672, 1119, 116853, 13366, 13367, 5755, 1826, 104435, 57102, 39554, 162, 2280, 967, 2201, 104751, 5411, 9273, 95569, 1488, 1970, 3250, 29393, 1787, 19440, 2020, 124198, 46741, 2541, 4066, 41202, 3322, 12083, 57057, 58448, 63706, 116356, 17359, 53829, 11623, 10631, 33435, 63267, 30316, 7487, 19300, 12017, 673, 16924, 1971, 1486, 11446, 170, 1683, 142, 2476, 42130, 11911, 1503, 33805, 11962, 56855, 29460, 1691, 559, 1892, 66492, 40779, 17652, 4596, 1615, 1120, 1322, 38977, 36376, 17140, 11003, 107073, 122736, 30910, 11720, 32522, 1457, 16111, 3143, 22970, 4993, 26152, 11715, 19744, 29913, 16937, 16934, 39911, 82215, 11803, 40062, 3178, 95685, 42622, 12239, 57095, 56414, 856, 44904, 7985, 13269, 52061, 22226, 59092, 38314, 64386, 24905, 1953, 94808, 3882, 58745, 79399, 96305, 1329, 63979, 109791, 6

In [31]:
print(len(cluster1))
print(len(cluster2))
print(len(cluster3))
print(len(cluster4))
print(len(cluster5))

24457
340
23562
374
23147


In [32]:
def getCommonNeighbors(G, cl1, cl2):
    x = set([])
    for i in cl1:
        x = x.union(set(G.adj[i]))
    #print('x: ', x)
    y = set([])
    for j in cl2:
        y = y.union(set(G.adj[j]))
    #print('y: ', y)
    return list(x.intersection(y))


In [33]:
c_g = getCommonNeighbors(G, cluster1, cluster2)
print(c_g)

[1543, 6153, 21015, 104473, 44570, 73762, 36899, 19497, 30764, 4152, 12353, 13897, 126538, 126539, 23118, 11863, 46173, 95332, 46696, 19053, 6256, 54387, 41075, 28275, 19067, 635, 37000, 21640, 25226, 25228, 18065, 28818, 1692, 160, 21665, 1189, 58022, 51879, 23209, 7342, 30384, 28849, 39098, 97476, 50886, 33991, 12999, 96972, 28880, 12508, 31453, 45814, 45825, 13062, 21775, 3855, 31000, 37662, 96543, 23842, 25891, 22828, 3890, 37174, 52036, 88395, 103251, 852, 6997, 2412, 5485, 367, 4982, 4990, 2943, 75661, 1934, 44944, 49555, 36259, 21929, 125356, 54722, 3011, 40389, 15311, 4564, 15835, 15838, 993, 44002, 11244, 54767, 10738, 12788, 33783]


In [34]:
print(len(c_g))

96


In [35]:
def fX(Cluster, s_AB):
    y = []
    for v in s_AB:
        x = []
        for u in Cluster:
            if (u, v) in G.edges():
                #print(str((u, v)) + ": " + str(AA1[(u, v)]))
                x.append(AA1[(u, v)])
        #print("#")
        y.append(np.mean(np.array(x)))
    return np.array(y)

In [36]:
y1 = fX(cluster1, c_g)

In [37]:
y2 = fX(cluster2, c_g)

In [38]:
def getSimiliarity(y1, y2):
    alpha = np.dot(y1.T, y2)
    beta = np.dot(y1.T, y1)
    gamma = np.dot(y2.T, y2)
    return alpha / np.sqrt((beta * gamma))

In [39]:
def getMainSimiliarity(cl1, cl2, s_AB):
    y1 = fX(cl1, s_AB)
    y2 = fX(cl2, s_AB)
    return getSimiliarity(y1, y2)

In [40]:
print(getMainSimiliarity(cluster1, cluster2, c_g))

-0.7115041611065441


In [41]:
def whereIsIt(main_clusters, u):
    n = main_clusters.shape[0]
    x = np.zeros(n)
    r = -1
    for i in range(n):
        if u in main_clusters[i]:
            r = i
    return r

In [42]:
print(whereIsIt(main_clusters, 17043))

2


In [43]:
def calcX(G, u):
    x = []
    for v in G.nodes():
        if ((v, u) in G.edges()):
            x.append(v)
    return np.array(x)

In [44]:
print(calcX(G, 6280))

[ 18840   2356   9447  29937  15520  30540  30413   2431    354  46023
   7974  44761  22658   6268  67663   5399  21333  17353  44722  68078
  52775    471  20093  20544  27969   7184  15544  29225  13945   1421
  21146  26555    204  21666  22076   6286  12580  21196  15216  69824
  20956  18225  51356  39238  28298  21250   3271  26726  64212  24715
    268  11615  20369  72152  63799  51891   5800  36111  33750  59258
  56414  25787  44760  56583  31483  19878  12170  29896  19767  80501
   4893  72279  36716    802  52812   6179  49437  16744  43943  24511
   8912  59862   2185  29032  47729  50008  22749  69847  35557  67314
  74747  58428  12761  50231   5124  58240  58147  17089  36299  30150
  30839  66269    331  72188  12081   6266  71961  66971  47796   9091
  57864  75320  16386  10124  27021 119919  34979  86545  38677  65911
  37042  37399  88570  56544  70609  72021  80734  75088  46285  31271
   7296  58134  75159  78028  80952  86573]


In [45]:
def whereAreThey(main_clusters, U):
    x = np.zeros(len(U))
    for i in range(len(x)):
        x[i] = whereIsIt(main_clusters, U[i])
    return x

In [46]:
print(whereAreThey(main_clusters, [6280, 24449,57345]))

[2. 4. 0.]


In [47]:
def signPrediction(G, AA1, main_clusters, s_AB, u, v, threshold):
    #if ((u, v) in G.edges()):
        #pass
    #else:
        u_c = whereIsIt(main_clusters, u)
        v_c = whereIsIt(main_clusters, v)
        S = calcX(G, v)
        S_c = whereAreThey(main_clusters, S)
        
        n = len(S)
        x = np.zeros(n)
        for i in range(n):
            x[i] = getMainSimiliarity(main_clusters[u_c], main_clusters[int(S_c[i])], s_AB)
        r = np.zeros(n)
        for i in range(n):
            r[i] = AA1[(S[i], v)]
        #print(x)
        #print(r)
        result = (np.dot(x.T, r)) / (np.sum(x))
        if result > threshold:
            return 1
        else:
            return -1

In [51]:
print(signPrediction(G, AA1, main_clusters, c_g,6, 307, 0))

-1


In [98]:
####################################################################

In [ ]:
zz = np.zeros(test.shape[0]
test1['sign prediction'] = test1.apply(lambda row : signPrediction(G, AA1, main_clusters, c_g, row['source'], row['target'], 0), axis = 1)
              

In [182]:
confusion_matrix(test['sign'], test['sign prediction'])

array([[1, 0],
       [1, 3]], dtype=int64)

In [183]:
accuracy_score(test['sign'], test['sign prediction'],  normalize=True, sample_weight=None)

0.8